## Dependencies

In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import pickle
import praw
import requests
import json
import time
import datetime
from tqdm import tqdm
import json

### List of Properties of a submission(post is often reffered as submission), I am manually eliminating few of them, bacuase flair of the submission is least (or not) dependent on them ie: Flair is rarely correlated to author name, or subreddit etc

#### Submission Properties: dict_keys(['all_awardings', 'allow_live_comments', 'author', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_text', 'author_flair_type', 'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video', 'link_flair_background_color', 'link_flair_css_class', 'link_flair_richtext', 'link_flair_template_id', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'pwls', 'removed_by_category', 'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler', 'steward_reports', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title', 'total_awards_received', 'url', 'whitelist_status', 'wls'])


#### Reference: 
- https://praw.readthedocs.io/en/latest/index.html
- https://github.com/pushshift/apia

#### creating a reddit app instance to collect post data from r/india submreddit

In [3]:
reddit = praw.Reddit(client_id='pw8WCM92ySUjsQ', client_secret='y0MJFMWBtHXMLW2-3B2upsU2jYQ', user_agent='reddit-scrap', username='macabdul9', password='Sudo$0#1')

#### We can get the submission either by id or url

In [4]:
subm = reddit.submission(id="fxqifi")

In [5]:
## test submission
subm.num_comments

126

#### Turn Around Time(TAT) for getting all comments of a post is way higher using praw with submission_id than pushshift.io api comment search below code snippets illustrate the same

In [7]:
# start_time = time.time()
# comment_ids = []
# count = 0
# for comment in subm.comments.list():
#     comment_ids.append(comment.id)
#     print(comment.body)
#     count+=1
# print("--- %s seconds ---" % (time.time() - start_time))
# print(count)

In [ ]:
# start_time = time.time()
# comments = getPushshiftSubData(','.join(comment_ids))
# count = 0
# for comment in comments:
#     print(comment['body'])
#     count +=1
# print("--- %s seconds ---" % (time.time() - start_time))
# print(count)

### Reddit API does not allow to scarp more than 1000 submissions per request hence scrapping large data using reddit api is not possible. Pushshift api https://github.com/pushshift/api allows to scrap unlimited amount of data

#### This function will receive url of a submission, num_commnents as arg and it will return the top 10 comment and mean comment score(upvote - downvote)

In [ ]:
def get_comments(url, num_comments):
    # call api using api wrapper
    subm = reddit.submission(url=url)
    comments_body = []
    sum_score = 0
    count = 0
    if num_comments > 0:
        for i, comment in enumerate(subm.comments.list()):

            # There may be some comments which has no body
            try : 
                comments_body.append(comment.body)
            except:
                comments_body.append('')

            # There exist some comments which has not given a score
            try:
                sum_score += comment.score
            except:
                sum_score += 0            
            count += 1
            # We only need 10 comments
            if (i+1)%10==0:
                break
    try:
        mean = sum_score/count
    except:
        mean = 0.0

    return " ".join(comments_body), mean

#### Function to call pushshift api and retrive the data


In [ ]:
def getPushshiftSubData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=2000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

#### Properties to extract 
- A custom property is a property derived from properties or can't be extracted using pushshift submission search endpoint

In [8]:
features = [
    'allow_live_comments', 'author', 'author_premium', 'can_mod_post', 'contest_mode', 'created_utc',
    'full_link', 'id', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_self', 'is_video',
    'link_flair_text', 'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18',
    'parent_whitelist_status', 'permalink', 'pinned', 'score', 'selftext', 'send_replies', 'spoiler',
    'stickied', 'title', 'total_awards_received', 'url'
]
custom_features = [
    'comments', 'mean_comment_score'
]

#### Function to extract the data 

In [ ]:
def collectSubData(subm):
    # extracting all information about a submission
    feature_values = [subm.get(key) for key in features]    
    comments, mean_comment_score = get_comments(subm.get('full_link'), subm.get('num_comments')) 
    feature_values.append(comments)
    feature_values.append(mean_comment_score)
    subStats[subm.get('id')] = feature_values

#### Query Parameters and Store Variables

In [ ]:
#Subreddit to query
sub='india'

#before and after dates
after = "1559347200"  # Sat, 01 Jun 2019 00:00:00 
before = "1561939199" # Sun, 30 Jun 2019 23:59:59  
query = "" # title should have either null string more 
subCount = 0
subStats = {}

#### Extracting the data from as per timestamp

In [ ]:
data = getPushshiftSubData(query, after, before, sub)# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    # print(len(data))
    if subCount % 1000 == 0:
        print(f'{subCount} submission collected')
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftSubData(query, after, before, sub)

print(len(data))

#### Write the json object into a pickle and/or store all the data into csv a file

In [ ]:
with open("Submissions-June-2019.pkl", "wb") as f:
    pickle.dump(subStats, f)

In [ ]:
df = pd.DataFrame(data=list(subStats.values()), columns = features + custom_features, index=None)

In [ ]:
df.to_csv("Submissions-June-2019.csv", index=False)

#### Notes
- To extract the large (or sufficient) amount of data above code [2. Data Acquisation] was executed in multiple python environments to decrease the amount of time taken to gather the data ie: To collect the submission of a month 2. [2. Data Acquisation] code was executed in seperate environment.
- On the other hand multiple reddit api instances (one api app instance for each running environment) also help to reduce the time to make API request faster(multiple running threads makes the request from same instance subtantially increases the waiting time)
- All submissions fromm Jan 1, 2019 to April 10, 2019 were collected for this task and stored into the csv files(pkl as well), one file for each month's data (1 file for March 1, 2020 to April 10, 2020)
- A reliable internet connection is required for uninterrupted exectution
